In [262]:
from deap import base
from deap import creator
from deap import tools
from deap import gp
from deap import algorithms
import operator
import math
import random
import numpy

In [263]:
# prepare data
def randomSequence(n):
    sequence = [random.uniform(-10,10) for i in range(n)]
    return sequence

X = []
for i in range(300):
    x = randomSequence(10)
    X.append(x)

X = numpy.array(X)
X = X.T
print(X.shape)

y = numpy.array(randomSequence(10))
print(y.shape)

(10, 300)
(10,)


In [264]:
def protectedDiv(left, right):
    try:
        return left/right
    except ZeroDivisionError:
        return 1
print(X.shape[1])
pset = gp.PrimitiveSet("MAIN", X.shape[1]) # the name of pset, and the number of input
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(numpy.cos, 1)
pset.addPrimitive(numpy.sin, 1)
_id = random.randint(1, 100)
print(_id)
pset.addEphemeralConstant("rand{}".format(_id), lambda: random.randint(-1,1))

300
21


In [265]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [266]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evaluate(individual):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    yp = func(*X.T)
    sqerrors = numpy.sqrt(numpy.nanmean((y-yp)**2))
    return sqerrors,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut,pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)

## Then, we decorate the mate and mutate method to limit the height of generated individuals. 
## This is done to avoid an important draw back of genetic programming : bloat. Koza in his book 
## on genetic programming suggest to use a max depth of 17.
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [267]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

In [268]:
random.seed(318)
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=40, 
                               stats=mstats, halloffame=hof, verbose=True)

SyntaxError: more than 255 arguments (<string>, line 1)

In [205]:
list(X[:,0])

[2.994256494115584,
 8.5536956042647923,
 -6.7883729942071724,
 -3.2544015788822218,
 -4.4429845148866809,
 3.247849173752666,
 8.0717688534910685,
 -0.35186965698921924,
 -0.45890187408747707,
 -2.5749910734298087]